In [14]:
import requests
from bs4 import BeautifulSoup

#送出request，並接收response
def RequestResponse(url):
    cookies = {"over18" : "1"}
    r = requests.get(url, cookies = cookies)
    r.encoding = "utf-8"
    respond = r.text
    
    return respond

In [15]:
#針對Beauty版
url = "https://www.ptt.cc/bbs/Beauty/index.html"
while True:    
    #以BeautifulSoup解析html
    soup_beauty = BeautifulSoup(RequestResponse(url), "html5lib")
    #print(soup.prettify())

    #尋找所有帖子存在的標籤
    contents = soup_beauty.find("div", class_ = "r-list-container action-bar-margin bbs-screen")
    #print(contents.prettify())

    #找到推文數>=11的帖子，class中hl f3代表推文數11~99，hl f1代表推爆
    post_popularity_list = contents.find_all("span", class_ = "hl f3" or "hl f1")
    print(post_popularity_list)

    #ptt第一頁下方會有公告，位置會在<div class="r-list-sep"></div>分隔線的下方
    #排除掉推文數高的公告
    bulletin_check = post_popularity_list[-1].find_all_previous("div", class_ = "r-list-sep")
    while bulletin_check != []:
        if len(list(post_popularity_list)) == 1:
            url = "https://www.ptt.cc{}".format(soup_beauty.find(string = "‹ 上頁").find_parent("a")["href"])
            print(url)
            break
        del post_popularity_list[-1]
        bulletin_check = post_popularity_list[-1].find_all_previous("div", class_ = "r-list-sep")
    
    if len(list(post_popularity_list)) == 1:
        print("This page ain't no good shit:( Trying previous page...")
        continue

    #找出推文數各自對應的帖子連結
    post_url_list = []
    for post_popularity in post_popularity_list:
        post_url_list.append(post_popularity.find_next("a")["href"])

    print(post_url_list)
    
    break


[<span class="hl f3">30</span>, <span class="hl f3">14</span>, <span class="hl f3">40</span>, <span class="hl f3">18</span>, <span class="hl f3">11</span>, <span class="hl f3">25</span>, <span class="hl f3">75</span>]
['/bbs/Beauty/M.1581696638.A.F13.html', '/bbs/Beauty/M.1581725717.A.9B5.html', '/bbs/Beauty/M.1581738006.A.63B.html', '/bbs/Beauty/M.1581743347.A.082.html', '/bbs/Beauty/M.1581747004.A.897.html', '/bbs/Beauty/M.1581760367.A.920.html']


In [19]:
import os
from PIL import Image
import time

#下載圖片，傳入檔名與副檔名(eg. xxxx.jpg)
def DownloadBeautyImage(img_id):
    
    # 決定要儲存的資料夾
    output_dir = './photos'
    
    # 假如資料夾不存在就新增一個資料夾
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 要request的圖片的url
    img_url = "https://i.imgur.com/{}".format(img_id)
    
    # 對圖片送出請求，設定為 stream 之後會跟網站建立通道
    with requests.get(img_url, stream = True) as r:
        # r.raise_for_status()會先檢查request是否成功，成功則不會有影響，失敗則會發起httperror
        try:
            r.raise_for_status()

            local_file_path = "./photos/{}".format(img_id)
            
            # 透過 loop 把檔案拆成很多 chunk 下載
            with open(local_file_path, "wb") as f:
                for chunk in r.iter_content(chunk_size = 8192):
                    if chunk:
                        f.write(chunk)
            
            # 此處才進行檔案格式分析，因為若在前面開啟，會把網頁的respond搶走，使loop不會接收到任何資料
            # 如果此處Image.open()打開的是r.raw也會因為loop把網頁的respond搶走，使Image.open()不會接收到任何資料
            # ...
            with Image.open(local_file_path) as img:
                img_ext = img.format.lower()

            new_local_file_path =  "{img_root}.{img_ext}".format(
                    img_root = os.path.splitext(local_file_path)[0], img_ext = img_ext)
            if local_file_path != new_local_file_path:
                try:
                    os.rename(local_file_path, new_local_file_path)
                except OSError as error:
                    if type(error) == FileExistsError:
                        os.remove(local_file_path)
                    print(error)
                    
                    return 0
                else:
                    print("Sucess")
        
        # request失敗
        except requests.exceptions.HTTPError as internet_error:
            print(internet_error)
            
            return 0
    
    return local_file_path
    

Start_time = time.time()    
    
for post_url in post_url_list:
    response = RequestResponse("http://www.ptt.cc{}".format(post_url))
    soup_beauty_post = BeautifulSoup(response, "html5lib")
    
    img_tag_list = soup_beauty_post.find(id = "main-content").findChildren("a", recursive = False)
    
    for img_tag in img_tag_list:
        
        # 取得所有圖片在第三方服務的 id
        if 'imgur' not in img_tag['href']:
            continue
        
        # 取出檔名與副檔名(eg. xxxx.jpg)
        img_id = img_tag["href"].split("/")[-1]

        # 呼叫自定義下載圖片的函式，傳入檔名與副檔名(eg. xxxx.jpg)，傳回所下載的檔案的路徑
        local_file_path = DownloadBeautyImage(img_id)
        
        # local_file_path == 0代表Error，印出失敗訊息並繼續下載
        if local_file_path == 0:
            print("Continue to download other images...")
            continue
        # 其他情況代表成功，印出成功訊息並繼續下載
        print(local_file_path)
        local_file_name = local_file_path.split("/")[-1]
        print("{local_file_name} has been saved in {local_file_path}".format(
            local_file_name = local_file_name, local_file_path = os.path.abspath(local_file_path)))
        
End_time = time.time()

print("total time consumed: {}".format(End_time - Start_time))

#第一次下載124s, 116s
#第二次下載，檢查檔案覆蓋160s, 123s

./photos/S7yX1Uo.gif
S7yX1Uo.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\S7yX1Uo.gif
./photos/4lAdfUW.gif
4lAdfUW.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\4lAdfUW.gif
./photos/oalbrQy.gif
oalbrQy.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\oalbrQy.gif
[WinError 183] 當檔案已存在時，無法建立該檔案。: './photos/67tKYda.jpg' -> './photos/67tKYda.jpeg'
Continue to download other images...
./photos/BTyiJJQ.gif
BTyiJJQ.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\BTyiJJQ.gif
./photos/r8j7e2V.gif
r8j7e2V.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\r8j7e2V.gif
./photos/WLF8fbk.gif
WLF8fbk.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\WLF8fbk.gif
./photos/102D2LD.gif
102D2LD.gif has been saved in D:\台大\Github\1st-PyCrawlerMarathon\Homework\photos\102D2LD.gif
./photos/ouOIDD4.gif
ouOIDD4.gif has been saved in D:\台大\Github\1st-PyCrawlerMarat

In [95]:
"""url = "https://i.imgur.com/9yk9sk8/88.jpg"

with requests.get(url, stream = True) as r:
    try:    
        r.raise_for_status()
            
        local_file_path = "./photos/{}".format(url.split("/")[-1])
        
        with open(local_file_path, "wb") as f:
            for chunk in r.iter_content(chunk_size = 8192):
                if chunk:
                    f.write(chunk)
                    
        with Image.open(local_file_path) as img:
            img_ext = img.format.lower()
        
        new_local_file_path =  "{img_root}.{img_ext}".format(
                img_root = os.path.splitext(local_file_path)[0], img_ext = img_ext)
        if local_file_path != new_local_file_path:
            try:
                os.rename(local_file_path, new_local_file_path)
            except OSError as e:
                if type(e) == FileExistsError:
                    os.remove(local_file_path)
                print(e)
            else:
                print("Sucess")
                print(os.path.abspath(new_local_file_path))
    
    except requests.exceptions.HTTPError as internet_error:
        print(internet_error)
        print("Continue to download other image...")"""

404
404 Client Error: Not Found for url: https://imgur.com/9yk9sk8/88
